In [2]:
import pandas as pd
import re
import ipywidgets as widgets

from datetime import datetime
from Corpus import Corpus
from Document import Document
from SearchEngine import SearchEngine
from IPython.display import display

In [3]:
#Question 1.1
file_path = "csv/discours_US.csv"
df = pd.read_csv(file_path, sep='\t')

In [4]:
#Question 1.2
auteurs = df["speaker"].value_counts()
print(auteurs)

speaker
CLINTON    93
TRUMP      71
Name: count, dtype: int64


In [5]:
#Question 1.3
corpus = Corpus("Discours US")
for _, row in df.iterrows():
    auteur = row["speaker"]
    text = row["text"]

    phrases = re.split(r'[.!?]+', text)

    for i, phrase in enumerate(phrases):
        phrase = phrase.strip()
        if len(phrase) > 0:
            doc = Document(
                titre = f"Discours_US_{i}",
                auteur = auteur,
                date = datetime.now(),
                url = "discours_us",
                texte = phrase
            )
            corpus.addDocument(doc)

print(corpus)
print(corpus.afficherAuteurs)


Corpus 'Discours US' :
 - 33836 documents
 - 2 auteurs
<bound method Corpus.afficherAuteurs of Corpus 'Discours US' :
 - 33836 documents
 - 2 auteurs>


In [6]:
#Question 1.4
df = df.rename(columns={"text": "texte"}) #Pour éviter une erreur car df ne contient pas texte mais text

keyword = "america"
results = corpus.search(df, keyword)

print(f"Résultats pour le mot {keyword} :")
for r in results:
    print(r)

df_concorde = corpus.concorde(df, keyword)
df_concorde.head()

Résultats pour le mot america :
('America', 105, 152, 'ning for president. Americans have fought their')
('America', 238, 285, 't the top. Everyday Americans need a champion a')
('America', 401, 448, "amilies are strong, America is strong. So I'm h")
('America', 2147, 2194, 'nk we all know that Americans have come back fr')
('America', 2264, 2311, ' better off because American families have basi')
('America', 2613, 2660, 'omething wrong when American workers keep getti')
('America', 3650, 3697, 'ecause I think that Americans and their familie')
('America', 6000, 6047, 'who goes to bat for Americans in four big areas')
('America', 7687, 7734, 'the backbone of the American economy. Here in N')
('America', 8845, 8892, 'ting and growing in America again. We have stal')
('America', 9633, 9680, ' and experiences of American workers and busine')
('America', 11996, 12043, ' times, and I think Americans have done everyth')
('America', 20540, 20587, 'ems we face is that American worker productiv

,contexteGauche,mot,contexteDroit
0,ning for president.,America,ns have fought their
1,t the top. Everyday,America,ns need a champion a
2,"amilies are strong,",America,is strong. So I'm h
3,nk we all know that,America,ns have come back fr
4,better off because,America,n families have basi


In [7]:
#Question 2.1, 2.2 et test de la 2.3
search_engine = SearchEngine(corpus)

#Requete avec 1 mot clé
# keyword = "america"
# results = search_engine.search(keyword, nResults=5)
# results

#Requete avec plusieurs mots clés
keyword = "america democracy"
results = search_engine.search(keyword, nResults=5)
results

Parcours des documents:   0%|          | 5/33836 [00:00<00:38, 868.13it/s]


,score,titre,auteur,date,url
0,0.5996,Discours_US_449,CLINTON,2025-12-17 18:22:35.436054,discours_us
1,0.5899,Discours_US_114,CLINTON,2025-12-17 18:22:35.337521,discours_us
2,0.5024,Discours_US_111,TRUMP,2025-12-17 18:22:35.412705,discours_us
3,0.4964,Discours_US_45,TRUMP,2025-12-17 18:22:35.465958,discours_us
4,0.4938,Discours_US_90,CLINTON,2025-12-17 18:22:35.474037,discours_us


In [9]:
#Partie 3
titreLabel = widgets.Label("Moteur de recherche")

motClesText = widgets.Text(
    value = '',
    placeholder = 'Ecrivez votre mot',
    description = 'Mots clés :',
)

auteursText = widgets.Text(
    value = '',
    placeholder = 'Nom de l\'auteur',
    description = 'Auteur :'
)

dateText = widgets.Text(
    value = '',
    placeholder = 'AAAA-MM-JJ',
    description = 'Date'
)

slider = widgets.IntSlider(
    value = 50,
    min = 1,
    max = 5000,
    step = 1,
    description = 'Nombre d\'articles à extraire'
)

resultsOutput = widgets.Output()

boutonSearch = widgets.Button(
    description = "Rechercher",
    button_style = "info"
)

ligne_widgets = widgets.VBox([motClesText, slider])
interface = widgets.VBox([titreLabel, ligne_widgets, auteursText, dateText, resultsOutput, boutonSearch])
display(interface)

def clique_bouton(button):
    with resultsOutput:
        resultsOutput.clear_output()
        keyword = motClesText.value
        nResults = slider.value
        auteurs = auteursText.value
        date = dateText.value
        results = search_engine.search(keyword, nResults=nResults, auteur=auteurs, date=date)
        if not results.empty:
            for i, doc in enumerate(results.itertuples(index=False), 1):
                print(f"{i}. {doc}")
        else:
            print("Aucun résultat trouvé.")

boutonSearch.on_click(clique_bouton)